## Load CSV

In [ ]:
import pandas as pd
from transformers import BertTokenizer
import torch
from torch.utils.data import Dataset, DataLoader, random_split

train_file = 'data/train.csv'
test_file = 'data/test.csv'
sample_file = 'data/sample.csv'

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)
sample_df = pd.read_csv(sample_file)

train_df['text'] = train_df['text'].apply(str)
test_df['text'] = test_df['text'].apply(str)
sample_df = pd.read_csv('sample.csv')

## Dataset

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self, df, tokenizer):
    self.tokenizer = tokenizer
    self.data = df
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    text = self.data['text'][index]
    label = self.data['label'][index] if 'label' in self.data.columns else None

    # Tokenize the text
    encoded_dict = self.tokenizer.encode_plus(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    input_ids = encoded_dict['input_ids'][0]
    attention_mask = encoded_dict['attention_mask'][0]
    labels = encoded_dict['labels'] if 'labels' in encoded_dict else None

    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}